In [2]:
import pandas as pd
import numpy as np

In [3]:
df1 = pd.read_csv('data/combined_data_1.txt', header=None, names=['cust_id', 'rating'], usecols=[0,1])
# df2 = pd.read_csv('data/combined_data_2.txt', header=None, names=['cust_id', 'rating'], usecols=[0,1])
# df3 = pd.read_csv('data/combined_data_3.txt', header=None, names=['cust_id', 'rating'], usecols=[0,1])
# df4 = pd.read_csv('data/combined_data_4.txt', header=None, names=['cust_id', 'rating'], usecols=[0,1])

# df = df1.append([df2])
df = df1
df.shape

(24058263, 2)

In [4]:
movies_data = pd.read_csv('data/movie_titles.csv', encoding="ISO-8859-1", header=None, names=['movie_id', 'year', 'name'])
movies_data.shape

(17770, 3)

In [5]:
df.index = np.arange(0, len(df))

In [9]:
df_nan = pd.DataFrame(pd.isnull(df.rating))
df_nan = df_nan[df_nan['rating'] == True]
df_nan = df_nan.reset_index()

movie_np = []
movie_id = 1

for i, j in zip(df_nan['index'][1:], df_nan['index'][:-1]):
    if i - j - 1 < 0:
        print(i, j)
    temp = np.full((1, i-j-1), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1

last_rec = np.full((1, len(df) - df_nan.iloc[-1, 0] - 1), movie_id)
movie_np = np.append(movie_np, last_rec)

len(movie_np)

24053764

In [10]:
df = df[pd.notnull(df['rating'])]
df['movie_id'] = movie_np.astype(int)
df['cust_id'] = df['cust_id'].astype(int)
df.shape

<ipython-input-10-7d553812af45>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['movie_id'] = movie_np.astype(int)
<ipython-input-10-7d553812af45>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cust_id'] = df['cust_id'].astype(int)


(24053764, 3)

In [11]:
df.head()

,cust_id,rating,movie_id
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
5,823519,3.0,1


In [9]:
f = ['count', 'mean']
df_movie_summ = df.groupby('movie_id')['rating'].agg(f)
df_movie_summ.index = df_movie_summ.index.map(int)
movie_thresh = round(df_movie_summ['count'].quantile(0.7), 0)
drop_movie_list = df_movie_summ[df_movie_summ['count'] < movie_thresh].index

df_cust_summ = df.groupby('cust_id')['rating'].agg(f)
df_cust_summ.index = df_cust_summ.index.map(int)
cust_thresh = round(df_cust_summ['count'].quantile(0.7), 0)
drop_cust_list = df_cust_summ[df_cust_summ['count'] < cust_thresh].index

In [10]:
print(f'Original shape: {df.shape}')
df_cleaned = df[~df['movie_id'].isin(drop_movie_list)]
df_cleaned = df_cleaned[~df_cleaned['cust_id'].isin(drop_cust_list)]
print(f'New shape: {df_cleaned.shape}')
df_cleaned.head()

Original shape: (24053764, 3)
New shape: (17337458, 3)


,cust_id,rating,movie_id
696,712664,5.0,3
697,1331154,4.0,3
698,2632461,3.0,3
699,44937,5.0,3
700,656399,4.0,3


In [11]:
# Matrix
df_piv = pd.pivot_table(df_cleaned, values='rating', index='cust_id', columns='movie_id')
print(df_piv.shape)

(143458, 1350)


In [12]:
movies_data.set_index('movie_id', inplace=True)
movies_data.head()

,year,name
movie_id,,
1,2003.0,Dinosaur Planet
2,2004.0,Isle of Man TT 2004 Review
3,1997.0,Character
4,1994.0,Paula Abdul's Get Up & Dance
5,2004.0,The Rise and Fall of ECW


## Matrix Factorization

In [13]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [15]:
reader = Reader()
data = Dataset.load_from_df(df_cleaned[['cust_id', 'movie_id', 'rating']][:], reader)
svd = SVD()
# cross_validate(svd, data, measures=['RMSE', 'MAE'], n_jobs=4)
trainset = data.build_full_trainset()
svd.fit(trainset)

In [17]:
df_785314 = df[(df['cust_id'] == 785314) & (df['rating'] == 5)]
df_785314 = df_785314.set_index('movie_id')
df_785314 = df_785314.join(movies_data)['name']
df_785314

movie_id
57                                            Richard III
175                                        Reservoir Dogs
311                                               Ed Wood
329                                                 Dogma
331                                           Chasing Amy
395                                         Captain Blood
788                                                Clerks
798                                                  Jaws
907                                       Animal Crackers
985                                             The Mummy
1552                                          Black Adder
1905    Pirates of the Caribbean: The Curse of the Bla...
2000                          Four Weddings and a Funeral
2122                                 Being John Malkovich
2342                                        Super Size Me
2360                                  Singin' in the Rain
2430                           Alien: Collector's Edition
2443 

In [18]:
user_785314 = movies_data.copy()
user_785314 = user_785314.reset_index()
user_785314 = user_785314[~user_785314['movie_id'].isin(drop_movie_list)]

In [19]:
user_785314['estimate_score'] = user_785314['movie_id'].apply(lambda x: svd.predict(785314, x).est)
user_785314 = user_785314.drop('movie_id',  axis=1)
user_785314 = user_785314.sort_values('estimate_score', ascending=False)
user_785314.head(10)

,year,name,estimate_score
1551,1983.0,Black Adder,4.967895
3149,2000.0,Monty Python: The Life of Python,4.892328
2217,1999.0,Black Adder Back & Forth,4.826158
4114,1999.0,The Simpsons: Bart Wars,4.755125
3045,1990.0,The Simpsons: Treehouse of Horror,4.739599
2171,1991.0,The Simpsons: Season 3,4.733137
1512,1962.0,The Twilight Zone: Vol. 42,4.712430
67,2004.0,Invader Zim,4.700669
3443,2004.0,Family Guy: Freakin' Sweet Collection,4.687472
1019,1989.0,The Simpsons: Season 1,4.682344


## Recommend with Pearson R

In [21]:
def recommend(movie_title, min_count):
    print(f'Movie: {movie_title}')
    print('Top 10 movie recommendations')
    # get movie index
    i = int(movies_data.index[movies_data['name'] == movie_title][0])
    target = df_piv[i]
    similar_target = df_piv.corrwith(target)
    corr_target = pd.DataFrame(similar_target, columns=['PearsonR'])
    corr_target.dropna(inplace=True)
    corr_target = corr_target.sort_values('PearsonR', ascending=False)
    corr_target.index = corr_target.index.map(int)
    corr_target = corr_target.join(movies_data).join(df_movie_summ)[['PearsonR', 'name', 'count', 'mean']]
    print(corr_target[corr_target['count'] > min_count][:10].to_string(index=False))

recommend('Batman Begins', 0)

Movie: Batman Begins
Top 10 movie recommendations
 PearsonR                                                  name  count     mean
 1.000000                                         Batman Begins  54922 4.236699
 0.369210 Batman: The Animated Series: Tales of the Dark Knight   2432 3.583059
 0.335256                                      X2: X-Men United  98720 3.932202
 0.325970                    Batman Beyond: Return of the Joker   3704 3.604752
 0.321018                                             Unleashed   8299 3.659718
 0.298462                                  The Bourne Supremacy 137170 3.909958
 0.294751                              Batman Beyond: The Movie   2614 3.726855
 0.293496                          Batman: Mask of the Phantasm   2823 3.767977
 0.283511                                        Justice League   3591 3.710944
 0.282093                    The Librarian: Quest for the Spear   2071 3.522936


## NCF

In [15]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Dense, Embedding, StringLookup, Concatenate, Dropout
import tensorflow_recommenders as tr

In [13]:
df.dtypes

cust_id       int32
rating      float64
movie_id      int32
dtype: object

In [35]:
class Ranking(keras.Model):
    def __init__(self, userid, movieid, embedding_size):
        super(Ranking, self).__init__()

        # user
        input = Input(shape=(), dtype=tf.string)
        x = StringLookup(vocabulary=userid, mask_token=None)(input)
        output = Embedding(input_dim=len(userid)+1, output_dim=embedding_size, name='embedding')(x)
        self.user_model = keras.Model(inputs=input, outputs=output, name='user')

        # item
        input = Input(shape=(), dtype=tf.string)
        x = StringLookup(vocabulary=movieid, mask_token=None)(input)
        output = Embedding(input_dim=len(movieid)+1, output_dim=embedding_size, name='embedding')(x)
        self.item_model = keras.Model(inputs=input, outputs=output, name='item')

        # rating
        user_input = Input(shape=(embedding_size,), name='user_emb')
        item_input = Input(shape=(embedding_size,), name='item_emb')
        x = Concatenate(axis=1)([user_input, item_input])
        x = Dense(256, activation='relu')(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(64, activation='relu')(x)
        x = Dense(32, activation='relu')(x)
        output = Dense(1)(x)
        self.rating_model = keras.Model(inputs={'cust_id': user_input, 'movie_id': item_input}, outputs=output, name='rating_model')

    def call(self, inputs):
        user_emb = self.user_model(inputs['cust_id'])
        item_emb = self.item_model(inputs['movie_id'])
        prediction = self.rating_model({'cust_id': user_emb, 'movie_id': item_emb})

        return prediction

class GMF(tr.models.Model):
    def __init__(self, userid, movieid, embedding_size):
        super().__init__()
        self.ranking_model = Ranking(userid, movieid, embedding_size)
        self.task = tr.tasks.Ranking(loss=keras.losses.MeanSquaredError(), metrics=[keras.metrics.RootMeanSquaredError()])

    def call(self, features):
        return self.ranking_model({'cust_id': features['cust_id'], 'movie_id': features['movie_id']})

    def compute_loss(self, features, training=False):
        label = features.pop('rating')
        return self.task(labels=label, predictions=self.ranking_model(features))

In [39]:
def df_ds(df):
    ds = tf.data.Dataset.from_tensor_slices((dict(df[['cust_id', 'movie_id']]), df['rating']))
    ds = ds.map(lambda x, y: {'cust_id': x['cust_id'], 'movie_id': x['movie_id'], 'rating': y})
    return ds.batch(1024)

In [40]:
rating_df = df.copy()
rating_df['cust_id'] = rating_df['cust_id'].astype(str)
rating_df['movie_id'] = rating_df['movie_id'].astype(str)

In [41]:
rating_df.head()

,cust_id,rating,movie_id
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
5,823519,3.0,1


In [42]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(rating_df, train_size=0.8)
train, test = df_ds(train), df_ds(test)

In [43]:
users = rating_df['cust_id'].unique()
movies = rating_df['movie_id'].unique()
users.shape, movies.shape

((470758,), (4499,))

In [44]:
embedding_size = 100
model = GMF(users.astype(str), movies.astype(str), embedding_size)
model.compile(optimizer=keras.optimizers.Adagrad(learning_rate=0.01))

In [45]:
model.fit(train, epochs=20, verbose=1)

Epoch 1/20
18793/18793 [==============================] - 101s 5ms/step - root_mean_squared_error: 1.0204 - loss: 1.0412 - regularization_loss: 0.0000e+00 - total_loss: 1.0412
Epoch 2/20
 5513/18793 [=======>......................] - ETA: 1:09 - root_mean_squared_error: 1.0058 - loss: 1.0117 - regularization_loss: 0.0000e+00 - total_loss: 1.0117

KeyboardInterrupt: 